In [3]:
import numpy as np

In [4]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [5]:
a = sample_Z(3,4)

In [6]:
np.random.uniform(-1., 1., size=[3, 4])

array([[-0.32981263,  0.05004392,  0.47627667,  0.13431565],
       [-0.16194942,  0.22798318, -0.39744505,  0.55331022],
       [ 0.23684063, -0.35169532, -0.34942995,  0.51977966]])

In [7]:
a

array([[-0.27464269,  0.71796398,  0.48779483, -0.85104084],
       [ 0.55720824, -0.19462209, -0.47613508, -0.33506048],
       [ 0.40081993, -0.84522717,  0.80286316,  0.90950975]])

In [8]:
import tensorflow as tf 
from tensorflow.contrib.layers import fully_connected
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [8]:
he_init = tf.contrib.layers.variance_scaling_initializer()

def dnn(inputs, n_hidden_layers=1, n_neurons=128, pre_name=None, activation=tf.nn.elu, 
        initializer=he_init, reuse=None):
    if pre_name == None:
        pre_name = ""
    with tf.name_scope("dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation, \
                    kernel_initializer=initializer, name="%shidden%d" % (pre_name, layer + 1), reuse=reuse)
    return inputs

In [10]:
n_D_inputs = 28*28
n_G_inputs = 100

n_D_outputs = 1
n_G_outputs = 28*28

In [27]:
reset_graph()

In [28]:
#该函数将给出权重初始化的方法
def variable_init(size):
    in_dim = size[0]

    #计算随机生成变量所服从的正态分布标准差
    w_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=w_stddev)

In [31]:
#定义输入矩阵的占位符，输入层单元为784，None代表批量大小的占位，X代表输入的真实图片。占位符的数值类型为32位浮点型
X = tf.placeholder(tf.float32, shape=[None, 784])

#定义判别器的权重矩阵和偏置项向量，由此可知判别网络为三层全连接网络
D_W1 = tf.Variable(variable_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(variable_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

#定义生成器的输入噪声为100维度的向量组，None根据批量大小确定
Z = tf.placeholder(tf.float32, shape=[None, 100])

#定义生成器的权重与偏置项。输入层为100个神经元且接受随机噪声，
#输出层为784个神经元，并输出手写字体图片。生成网络根据原论文为三层全连接网络
G_W1 = tf.Variable(variable_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(variable_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [30]:
Z = tf.placeholder(tf.float32, shape=(None, n_G_inputs), name="Z")
X = tf.placeholder(tf.float32, shape=(None, n_D_inputs), name="X")

In [32]:
def generator(Z):
    # g_latent_out = dnn(Z, pre_name="g_")
    # g_logits = tf.layers.dense(g_latent_out, n_G_outputs, kernel_initializer=he_init, name="g_logits")
    # g_proba = tf.nn.sigmoid(g_logits, name="g_proba")
    G_h1 = tf.nn.relu(tf.matmul(Z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    g_logits  = G_log_prob
    g_proba = G_prob
    # return g_logits, g_proba
    return G_prob

In [33]:
def discriminator(X, reuse=None):
    # d_latent_out = dnn(X, pre_name="d_",reuse=reuse)
    # d_logits = tf.layers.dense(d_latent_out, n_D_outputs, kernel_initializer=he_init, name="d_logits", reuse=reuse)
    # d_proba = tf.nn.sigmoid(d_logits)
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    d_logits = D_logit
    d_proba = D_prob
    # return d_logits, d_proba
    return D_prob, D_logit

In [11]:
"""
we need the variables for each layers of models
g_proba is G(Z)
"""
g_logits, g_proba = generator(Z)
d_logits_data, d_proba_data = discriminator(X)

'\nwe need the variables for each layers of models\ng_proba is G(Z)\n'

In [12]:
d_logits_g, d_proba_g = discriminator(g_proba, reuse=True)

In [1]:
Variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
theta_G = Variables[:4]
theta_D = Variables[4:]

NameError: name 'tf' is not defined

In [34]:
#输入随机噪声z而输出生成样本
g_proba = generator(Z)

#分别输入真实图片和生成的图片，并投入判别器以判断真伪
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(g_proba)

#以下为原论文的判别器损失和生成器损失，但本实现并没有使用该损失函数
# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

# 我们使用交叉熵作为判别器和生成器的损失函数，因为sigmoid_cross_entropy_with_logits内部会对预测输入执行Sigmoid函数，
#所以我们取判别器最后一层未投入激活函数的值，即D_h1*D_W2+D_b2。
#tf.ones_like(D_logit_real)创建维度和D_logit_real相等的全是1的标注，真实图片。
loss_data = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
loss_g = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))

#损失函数为两部分，即E[log(D(x))]+E[log(1-D(G(z)))]，将真的判别为假和将假的判别为真
loss_D = loss_data + loss_g

#同样使用交叉熵构建生成器损失函数
loss_G = loss_g

#定义判别器和生成器的优化方法为Adam算法，关键字var_list表明最小化损失函数所更新的权重矩阵
training_op_d = tf.train.AdamOptimizer().minimize(loss_D, var_list=theta_D)
training_op_g = tf.train.AdamOptimizer().minimize(loss_G, var_list=theta_G)

In [13]:
xentropy_g = tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_g, labels=tf.ones_like(d_logits_g))
loss_g = tf.reduce_mean(xentropy_g, name="loss_g")
xentropy_data = tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_data, labels=tf.ones_like(d_logits_data))
loss_data = tf.reduce_mean(xentropy_data, name="loss_data")

In [15]:
ones = tf.ones_like([[1,-1,-1],[1,1,-1]])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ones_val = sess.run(ones)

In [14]:
loss_D = loss_data + loss_g
loss_G = tf.reduce_mean(xentropy_g, name="loss_G")

In [15]:
learning_rate = 0.01

In [35]:
training_op_d = tf.train.AdamOptimizer().minimize(loss_D, var_list=theta_D)
training_op_g = tf.train.AdamOptimizer().minimize(loss_G, var_list=theta_G)
# training_op_d = optimizer_d.minimize(loss_D, var_list=
# theta_D)
# training_op_g = optimizer_g.minimize(loss_G, var_list=theta_G)

In [36]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()

In [37]:
n_epochs = 20000
batch_size = 128
z_dim = 100

checkpoint_path = "/tmp/my_mnistGAN_model.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./my_mnistGAN_model"

In [38]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/Users/changxin/Documents/deeplearning/UnityPK/datasets/MNIST/")

Extracting /Users/changxin/Documents/deeplearning/UnityPK/datasets/MNIST/train-images-idx3-ubyte.gz
Extracting /Users/changxin/Documents/deeplearning/UnityPK/datasets/MNIST/train-labels-idx1-ubyte.gz
Extracting /Users/changxin/Documents/deeplearning/UnityPK/datasets/MNIST/t10k-images-idx3-ubyte.gz


Extracting /Users/changxin/Documents/deeplearning/UnityPK/datasets/MNIST/t10k-labels-idx1-ubyte.gz


In [39]:
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

if not os.path.exists('out/'):
    os.makedirs('out/')

In [40]:
#CREATE PHOTO
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


In [41]:
with tf.Session() as sess:
    sess.run(init)
    # if not os.path.exists(checkpoint_epoch_path):
    #     start_epoch = 0
    #     sess.run(init)
    # else:
    #     with open(checkpoint_epoch_path, 'rb') as f:
    #         start_epoch = int(f.read())
    #     print("Training was interrupted. Continuing at epoch", start_epoch)
    #     saver.restore(sess, checkpoint_path)
    i = 0 
    # for epoch in range(start_epoch, n_epochs):
    for epoch in range(n_epochs):
        
        
        X_batch, y_batch = mnist.train.next_batch(batch_size)
        # Z_batch = sample_Z(batch_size, z_dim)
        _, loss_D_val = sess.run([training_op_d, loss_D], feed_dict={X: X_batch, Z: sample_Z(batch_size, z_dim)})
        _, loss_G_val = sess.run([training_op_g, loss_G], feed_dict={Z: sample_Z(batch_size, z_dim)})
        
        if epoch % 2000 == 0:
            samples = sess.run(g_proba, feed_dict={Z: sample_Z(16, z_dim)})
            fig = plot(samples)
            plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            plt.close(fig)
            
        # if epoch % 5000 == 0:
        #     with open(checkpoint_epoch_path, 'wb') as f:
        #         saver.save(sess, checkpoint_path)
        #         f.write(b'%d' % (epoch + 1))
        
        if epoch % 2000 == 0:
            print('Iter: {}'.format(epoch))
            print('D loss: {:.4}'. format(loss_D_val))
            print('G_loss: {:.4}'.format(loss_G_val))

Iter: 0
D loss: 1.751
G_loss: 0.1543


Iter: 2000
D loss: 1.925e-06
G_loss: 5.106e-07


Iter: 4000
D loss: 2.502e-07
G_loss: 9.521e-08


Iter: 6000
D loss: 1.952e-07
G_loss: 2.842e-08


Iter: 8000
D loss: 1.321e-08
G_loss: 9.317e-09


Iter: 10000
D loss: 1.286e-08
G_loss: 3.043e-09


Iter: 12000
D loss: 1.342e-09
G_loss: 1.218e-09


In [3]:
import os
os.path.abspath(".")

'/Users/changxin/Documents/deeplearning/UnityPK'